In [20]:
import pandas as pd
import datetime as dt
import numpy as np
import indicators as ind
from RTLearner import RTLearner
from BagLearner import BagLearner

class StrategyLearner(object):

    def __init__(self, verbose = False, impact=0.0, commission=0.0):
        self.verbose = verbose
        self.impact = impact
        self.commission = commission
        self.N = 5  # Number of days to compute future return
        self.YBUY = 0.01  # Threshold for classifying 'BUY'
        self.YSELL = -0.01  # Threshold for classifying 'SELL'
        self.bag_learner = BagLearner(RTLearner, kwargs={"leaf_size": 5}, bags=20, boost=False, verbose=self.verbose)

    def add_evidence(self, symbol = "JPM", sd=dt.datetime(2008,1,1), ed=dt.datetime(2009,12,31), sv = 100000):
        df = ind.get_indicators(sd=sd, ed=ed, symbol=symbol)
        prices = df[symbol]
        indicators = df.drop(columns=[symbol])

        ret = (prices.shift(-self.N) / prices) - 1.0

        Y = pd.Series(index=prices.index)
        Y[ret > self.YBUY] = 1  # LONG
        Y[ret < self.YSELL] = -1  # SHORT
        Y[(ret >= self.YSELL) & (ret <= self.YBUY)] = 0  # CASH

        X = indicators.iloc[:-self.N]
        Y = Y.iloc[:-self.N]

        self.bag_learner.add_evidence(X.values, Y.values)

    def testPolicy(self, symbol = "JPM", sd=dt.datetime(2010,1,1), ed=dt.datetime(2011,12,31), sv = 100000):
        df = ind.get_indicators(sd, ed, symbol)
        df.index.name = 'Date'
        prices = df[symbol]
        indicators = df.drop(columns=[symbol])

        X = indicators.values
        Y_pred = self.bag_learner.query(X)

        # Initialize orders with None
        orders = pd.DataFrame(index=indicators.index, data={'Symbol': None, 'Order': None, 'Shares': None})
        orders.index.name = 'Date'
        holdings = 0

        # Add 'SELL 0' order for the first trading day
        first_trading_day = indicators.index.min()
        orders.loc[first_trading_day, 'Symbol'] = symbol
        orders.loc[first_trading_day, 'Order'] = 'SELL'
        orders.loc[first_trading_day, 'Shares'] = 0

        for i in range(len(indicators)):
            order_type = None
            order_amt = 0
            if Y_pred[i] > 0.5:  # 'BUY'
                if holdings < 0:
                    order_type = 'BUY'
                    order_amt = 2000  # Reverse position from -1000 to +1000
                    holdings += order_amt
                elif holdings == 0:
                    order_type = 'BUY'
                    order_amt = 1000  # Establish new long position
                    holdings += order_amt
            elif Y_pred[i] < -0.5:  # 'SELL'
                if holdings > 0:
                    order_type = 'SELL'
                    order_amt = 2000  # Reverse position from +1000 to -1000
                    holdings -= order_amt
                elif holdings == 0:
                    order_type = 'SELL'
                    order_amt = 1000  # Establish new short position
                    holdings -= order_amt

            if order_type:
                orders.loc[indicators.index[i], 'Symbol'] = symbol
                orders.loc[indicators.index[i], 'Order'] = order_type
                orders.loc[indicators.index[i], 'Shares'] = order_amt

        # Add 'BUY 0' order for the last trading day
        last_trading_day = indicators.index.max()
        orders.loc[last_trading_day, 'Symbol'] = symbol
        orders.loc[last_trading_day, 'Order'] = 'BUY'
        orders.loc[last_trading_day, 'Shares'] = 0

        # Drop all rows with no order
        orders.dropna(inplace=True)

        return orders




    def author(self):
        return 'ydeng335'

if __name__ == "__main__":
    print("One does not simply think up a strategy")


One does not simply think up a strategy


In [21]:
sl=StrategyLearner()
sl.add_evidence()
df=sl.testPolicy()

/tmp/ipykernel_186189/2200011257.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Y = pd.Series(index=prices.index)


In [4]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import ManualStrategy as ms
from StrategyLearner import StrategyLearner
from marketsimcode import compute_portvals
from util import get_data

def benchmark_orders(symbol, sd, ed):
    prices = get_data([symbol], pd.date_range(sd, ed))
    first_trading_day = prices.index.min()
    benchmark_orders = pd.DataFrame(data={'Symbol': [symbol],'Order': ['BUY'],  'Shares': [1000]}, index=[first_trading_day])
    benchmark_orders.index.name = 'Date'
    last_trading_day = prices.index.max()
    benchmark_orders.loc[last_trading_day] = [symbol, 'BUY', 0]
    # Fill NaN
    benchmark_orders.fillna(method='ffill', inplace=True)
    benchmark_orders.fillna(method='bfill', inplace=True)
    
    return benchmark_orders

def plot_chart(sd, ed, symbol, portvals_ms, portvals_sl, portvals_benchmark, title, filename):
    # Normalize portfolio values
    portvals_ms = portvals_ms / portvals_ms.iloc[0]
    portvals_sl = portvals_sl / portvals_sl.iloc[0]
    portvals_benchmark = portvals_benchmark / portvals_benchmark.iloc[0]

    plt.figure(figsize=(10, 5))
    plt.plot(portvals_ms.index, portvals_ms, color='red', label='Manual Strategy')
    plt.plot(portvals_sl.index, portvals_sl, color='blue', label='Strategy Learner')
    plt.plot(portvals_benchmark.index, portvals_benchmark, color='green', label='Benchmark')
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Normalized Portfolio Value')
    plt.legend(loc='best')
    plt.grid()
    plt.savefig(f'images/{filename}', dpi=300)
    plt.close()


def run_experiment(in_sample_sd, in_sample_ed, out_sample_sd, out_sample_ed, symbol="JPM"):
    sv = 100000  # Start value of each portfolio

    # In-sample period
    # Manual Strategy
    trades_ms_in_sample = ms.testPolicy(symbol, in_sample_sd, in_sample_ed, sv)
    portvals_ms_in_sample = compute_portvals(trades_ms_in_sample, start_val=sv)

    # Strategy Learner
    sl = StrategyLearner()
    sl.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed, sv=sv)
    trades_sl_in_sample = sl.testPolicy(symbol, sd=in_sample_sd, ed=in_sample_ed, sv=sv)
    portvals_sl_in_sample = compute_portvals(trades_sl_in_sample, start_val=sv)

    # Benchmark
    benchmark_trades_in_sample = benchmark_orders(symbol, in_sample_sd, in_sample_ed)
    portvals_benchmark_in_sample = compute_portvals(benchmark_trades_in_sample, start_val=sv)

    plot_chart(in_sample_sd, in_sample_ed, symbol, portvals_ms_in_sample, portvals_sl_in_sample, portvals_benchmark_in_sample, "In-Sample Comparison", "InSampleComparison.png")

    # Out-of-sample period
    # Manual Strategy
    trades_ms_out_sample = ms.testPolicy(symbol, out_sample_sd, out_sample_ed, sv)
    portvals_ms_out_sample = compute_portvals(trades_ms_out_sample, start_val=sv)

    # Strategy Learner
    sl.add_evidence(symbol=symbol, sd=out_sample_sd, ed=out_sample_ed, sv=sv)
    trades_sl_out_sample = sl.testPolicy(symbol, sd=out_sample_sd, ed=out_sample_ed, sv=sv)
    portvals_sl_out_sample = compute_portvals(trades_sl_out_sample, start_val=sv)

    # Benchmark
    benchmark_trades_out_sample = benchmark_orders(symbol, out_sample_sd, out_sample_ed)
    portvals_benchmark_out_sample = compute_portvals(benchmark_trades_out_sample, start_val=sv)

    plot_chart(out_sample_sd, out_sample_ed, symbol, portvals_ms_out_sample, portvals_sl_out_sample, portvals_benchmark_out_sample, "Out-of-Sample Comparison", "OutOfSampleComparison.png")

def author():
    return "ydeng335"  # Change this to your user ID

def exp1run():
    run_experiment(in_sample_sd=dt.datetime(2008, 1, 1), 
                   in_sample_ed=dt.datetime(2009, 12, 31),
                   out_sample_sd=dt.datetime(2010, 1, 1), 
                   out_sample_ed=dt.datetime(2011, 12, 31),
                   symbol="JPM") 
    
if __name__ == "__main__":
    exp1run()

/mnt/f/CS7646/strategy_evaluation/StrategyLearner.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Y = pd.Series(index=prices.index)
/mnt/f/CS7646/strategy_evaluation/StrategyLearner.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Y = pd.Series(index=prices.index)
